# Resolving Strategies

* Example on how to use resolving strategies


In [3]:
import os
import json

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.resources import Dataset

In [1]:
import getpass
TOKEN = getpass.getpass()

In [4]:
endpoint = "https://staging.nise.bbp.epfl.ch/nexus/v1"
BUCKET = "dke/kgforge"
forge = KnowledgeGraphForge("../use-cases/prod-forge-nexus.yml", endpoint=endpoint, bucket=BUCKET, token=TOKEN)

In [6]:
case_insensitive = forge.resolve('Pv+', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH')

In [7]:
exact = forge.resolve('PV+', scope='ontology', strategy='EXACT_MATCH')

In [8]:
assert case_insensitive == exact